In [1]:
import numpy as np
import pickle
from scipy import spatial
from scipy import ndimage

dim = 100

In [2]:
with open('data.p', 'rb') as fp:
    embeddings_dict = pickle.load(fp)

In [3]:
def get_centers(hints):
    centers = []
    for hints_list in hints:
        embeddings = []
        if hints_list == []:
            centers.append(np.zeros(dim))
        else :
            for word in hints_list:
                embeddings.append(embeddings_dict[word])
            centers.append(np.sum(embeddings,axis=0)/len(embeddings))
    return centers
    
def guess_word_1(hints,new_hint):
    embedding = embeddings_dict[new_hint]
    D = []
    for word_list in hints:
        dist = 0
        for word in word_list :
            dist += spatial.distance.cosine(embeddings_dict[word], embedding)
        D.append(dist)
    return np.argmin(D)

def guess_word_2(hints_centers,new_hint):
    embedding = embeddings_dict[new_hint]
    D = []
    for rep in hints_centers:
        dist = spatial.distance.cosine(rep, embedding)
        D.append(dist)
    return np.argmin(D)

In [11]:
keys = ['fruits','clothes','technology','transport']

hints = [['orange','apple'],
         ['pants'],
         ['phone'],
         ['car']]

new_hint = 'banana'

In [12]:
print("New hint : ",new_hint)
print("Sum of errors : ",keys[guess_word_1(hints,new_hint)])
print("Words centers : ",keys[guess_word_2(get_centers(hints),new_hint)])

New hint :  banana
Sum of errors :  clothes
Words centers :  fruits
